# Introduction

## Goals
These are the following goals of this homework: 

(1) Gain a hands on experience of single-cell RNA-seq analysis using [`scanpy`](https://scanpy.readthedocs.io/en/stable/). 

Additionally, scanpy uses a custom data structure called [`Anndata`](https://anndata.readthedocs.io/en/latest/) - a quick run down on the data structure will also be useful for the homework.

(2) Exploration of trajectory detection algorithms such Palantir, scVelo and CellRank. 

## Data
The dataset we will explore is a  RNA dataset generated using CD34+ human bone marrow cells to characterize human hematopoiesis. Data is available as `Anndata` objects and there are two objects:
 1. `cd34_multiome_rna_data.h5ad`: RNA gene expression. Cell type information is included in this Anndata object
 2. `cd34_multiome_rna_velocyto_data.h5ad`: Spliced and unspliced read counts computed using ['velocyto'](http://velocyto.org).

# Import necessary packages

In [ ]:
import scanpy as sc
import scanpy.external as sce
import palantir
import scvelo as scv
import cellrank as cr

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
%matplotlib inline

# scRNA analysis

In [ ]:
FILES_PATH = "/Users/tgoel/Downloads/Classes/GENOME/Genome541/hw3/"

# Load RNA data
rna_ad = sc.read(FILES_PATH + 'cd34_multiome_rna_data.h5ad')
rna_ad

## Preprocess

This section deals with preprocessing and analysis of the data. The provided data is pre-filtered

In [ ]:
# Normalize and log transform using scanpy functions
sc.pp.normalize_per_cell(rna_ad) # sc.pp.normalize_total(rna_ad, target_sum=1)
sc.pp.log1p(rna_ad)

<b>`Questions`</b>: Why is normalization and log transformation necessary? How would skipping one or both cell types change the results?

Normalization and log transformation are important because they help improve the quality of the data. It is especially important here because there is a large skew in the distribution of total molecules (UMIs) per cell making expression values hard to compare across cells. Log transformation would help transform this highly skewed data into a more normal distribution, and normalization can help prevent outliers with larger feature values from dominating the analysis. Skipping one or both of these steps would make the data more difficult to analyze and interpret.

Given the high degree of noise in single-cell data, it is important to select informative features (genes) that best explain the data. This feature selection should be system agnostic and should not rely on prior biological knowledge. Normalized dispersion, a measure gene variability corrected for the mean is a good way to select informative genes. 

Drop-outs still pose an issue post feature selection for measuring distance between cells. PCA can be used a build a "meta-gene" representation of the data to measure disances between cells.

In [ ]:
# Use scanpy to select 1500 highly variable gene selection and perform PCA
sc.pp.highly_variable_genes(rna_ad, n_top_genes=1500)

In [ ]:
# Plot the normalized dispersion compared to mean and highlight the selected highly variable genes
sc.pl.highly_variable_genes(rna_ad)

In [ ]:
rna_ad.raw = rna_ad
#rna_ad = rna_ad[:, rna_ad.var.highly_variable]

A number of dimensionality reduction techniques are available for visualization and clustering of single-cell data. Here we will use `umap` for visualization and `leiden` for clustering of cells

In [ ]:
sc.pp.neighbors(rna_ad)
sc.tl.umap(rna_ad)
sc.tl.leiden(rna_ad)

In [ ]:
# Visualize leiden clusters and annotated cell types on the umap
sc.pl.umap(rna_ad, color=['leiden', 'celltype'], legend_loc='on data', ncols=2, wspace=0.3, hspace=0.3, frameon=False, size=50, alpha=0.5, palette='tab20')

## Differential analysis

Differential gene expression can be used to used significantly high (or low) genes in each cell type. 

In [ ]:
# Identify the top 5 differentially expressed genes for each cell type 
sc.tl.rank_genes_groups(rna_ad, 'leiden', method='wilcoxon')
pd.DataFrame(rna_ad.uns['rank_genes_groups']['names']).head(5)

In [ ]:
# Plot expression of marker genes per cell type and cluster
sc.pl.rank_genes_groups(rna_ad, n_genes=5)

In [ ]:
# Plot marker genes per cell type and cluster
sc.pl.rank_genes_groups_violin(rna_ad, n_genes=5)

In [ ]:
rna_ad.uns['rank_genes_groups']['names']

<b>`Exercise`</b>: Use the identified markers to associate each cluster to a cell type

In [ ]:
# Use the identified markers to associate each cluster with a cell type
marker_genes_dict = {
    'B cells': ['TUBA1B', 'HMGB2', 'BACH2'],
    'Mono': ['MPO'],
    'CLP': ['NEGR1'],
    'Ery': ['XACT'],
    'HMP': ['PRSS57', 'SPINK2'],
    'HSC': ['RPS24', 'NKAIN2'],
    'MEP': ['MED12L']
}

sc.pl.dotplot(rna_ad, marker_genes_dict, 'celltype', dendrogram=False)

# scRNA analysis without B cells

Create a new `anndata` object by removing the B cells from the data. We will use this data subset for trajectory analysis. Since the set of cells under consideration changed, one will need to recompute the highly variable genes, UMAP and clustering

In [ ]:
# Create an anndata object without B cells and name it as `rna_sub`
rna_sub = sc.AnnData(rna_ad.X[rna_ad.obs['celltype'] != 'B cells'], obs=rna_ad.obs[rna_ad.obs['celltype'] != 'B cells'])
rna_sub

In [ ]:
# Select 1500 highly variable genes and perform PCA
sc.tl.pca(rna_sub, svd_solver='arpack')
sc.pl.pca(rna_sub, color='celltype')

<b>`Question`</b>: How many genes are highly variable specifically in the subset of cells?

In [ ]:
# How many genes are highly variable specifically in the subset of cells?
sc.pp.highly_variable_genes(rna_sub)
rna_sub.var.highly_variable.sum()

In [ ]:
# Compure UMAP and leiden on the the reduced set of clusters
sc.pp.neighbors(rna_sub)
sc.tl.umap(rna_sub)
sc.tl.leiden(rna_sub)

In [ ]:
# Visualize leiden clusters and annotated cell types on the umap
sc.pl.umap(rna_sub, color=['leiden', 'celltype'], legend_loc='on data', ncols=2, wspace=0.3, hspace=0.3, frameon=False, size=50, alpha=0.5, palette='tab20')

The results should look similar to <img src="Users/tgoel/Downloads/Classes/GENOME/GENOME541/hw3/umap.png">


<b>`Question`</b>: Are your plots identical to the ones listed above? If not, why do you think they are not identical? How can I make sure to reproduce identical plots?

My plots are not identical to the ones listed above, but very similar. Maybe this is because the random seed is not set. To reproduce identical plots, I can set the random seed to 42 for example.

# Trajectory analysis - Palantir

We will perform trajectory analysis using [`Palantir`](https://www.nature.com/articles/s41587-019-0068-4). Palantir models differnitation as a Markov chain to estimate a pseudo-time ordering of cells and probability of each cell differentiating to any of the terminal states in the system. 

Palantir requires the specification of a start cell and works best when the terminal states are specified. For ease of use, the start and terminal states are specified below

In [ ]:
# Palantir start and terminal states 
terminal_states = pd.Series(dtype=object)
terminal_states['Ery'] = 'cd34_multiome_rep2#TACGGTTAGTTATTCC-1' # Erythroid terminal
terminal_states['Mono'] = 'cd34_multiome_rep2#CGCTCCATCGCTAGAT-1' # Monocyte terminal
terminal_states['Lymph'] = 'cd34_multiome_rep1#TTTAAGGTCCTTCTAG-1' # Lymphoid terminal
terminal_states['Mega'] = 'cd34_multiome_rep1#AGGCCCAGTGCATCGG-1' # Megakaryocyte terminal
start = 'cd34_multiome_rep1#AGGTCCGGTACGCGCA-1' # Hematopoietic stem cell

In [ ]:
# Compute the multi-scale space used by Palantir and visualize them
sc.pp.pca(rna_sub)
pca_projections = pd.DataFrame(rna_sub.obsm['X_pca'], index=rna_sub.obs_names)
dm_res = palantir.utils.run_diffusion_maps(pca_projections, n_components=5)
ms_data = palantir.utils.determine_multiscale_space(dm_res)

In [ ]:
# Run Palantir using the start and terminal states defined above
pr_res = palantir.core.run_palantirrun_palantir(ms_data, start, terminal_states=terminal_states)

In [ ]:
# Rename the columns of branch probabilities with cell types
flip = pd.Series(terminal_states.index, index=terminal_states.values)
pr_res.branch_probs.columns = flip[pr_res.branch_probs.columns].values

In [ ]:
# Visualize Palantir results
umap = pd.DataFrame(rna_sub.obsm['X_umap'], index=rna_sub.obs_names)
palantir.plot.plot_palantir_results.plot_palantir_results(pr_res, umap)

<b>`Exercise`<b/>
- Identify the subset of cells that specialize towards erythroid and monocyte fates and highlight them on the umap 
- Compute the top 5 differentially expressed transcription factors between these cells
    - A list of human transcription factors is available at [`AnimalTFDB`](http://bigd.big.ac.cn/databasecommons/database/id/8).
- Plot gene expression trends along the erythroid and monocyte lineage for these genes 
- Identify and plot the top 5 TFs that are most correlated with erythroid and monocyte fates 
- Are these genes the same genes you computed using differential expression 
- How many genes are differnt and reason why they are different 

# RNA velocity - scvelo

[`scVelo`](https://scvelo.readthedocs.io) is a tool to determine RNA velocities by leveraging the splicing dynamics information inherently present in RNA-seq data. The beauty of RNA velocity is that requires practically no prior biological knowledge of the system but is heavily reliant on splicing dynamics being sufficiently captured in the data.

scVelo requires the quantification of spliced and unspliced transcripts. These counts were computed using `velocyto` and are available as an `Anndata` object. This object can be loaded and merged with the scRNA Anndata object using: 

In [ ]:
# Load the velocyto results anndata and merge with rna
loom_ad = sc.read('/Users/tgoel/Downloads/Classes/GENOME/GENOME541/hw3/cd34_multiome_rna_velocyto_data.h5ad')
merged_ad = scv.utils.merge(rna_sub, loom_ad)

In [ ]:
scv.pp.filter_and_normalize(merged_ad, min_shared_counts=20, n_top_genes=2000)
scv.pp.moments(merged_ad, n_pcs=30, n_neighbors=30)

In [ ]:
# Compute the RNA velocities using the scvelo dynamical model
scv.tl.recover_dynamics(merged_ad)
scv.tl.velocity(merged_ad, mode='dynamical')
scv.tl.velocity_graph(merged_ad)

In [ ]:
# Plot the velocity stream with cells colored by cell type
scv.pl.velocity_embedding_stream(merged_ad, basis='umap')

<b>`Exercise`</b>:
- What are your key observations and how are they different from the Palantir results?
 Based on your observations, are the RNA velocity results consistent with known biology of hematopoiesis where hematpoietic stem cells (HSCs) differentiate to immune cell types?
- Plot the RNA splicing dynamics of key TFs you identified from Palantir analysis 
- Are the splicing dynamics consistent with the Palantir trajectories?

The RNA velocity results are not consistent with the Palantir results. The RNA velocity results are not consistent with known biology of hematopoiesis. The RNA splicing dynamics of key TFs are not consistent with the Palantir trajectories. The splicing dynamics are not consistent with the Palantir trajectories.

`Note`
scVelo is a fantastic algorithm and has been successfully used to characterize many biological systems.  However, the splicing dynamics do not seem to be informative for hematopoiesis (see [`here`](https://www.embopress.org/doi/full/10.15252/msb.202110282) for discussion as to why this might be the case) and therefore scVelo fails to recover the correct cell state dynamics. This is a nice illustration of how one size fits all almost never works with scRNA-seq data analysis and one should always pay special attention to what are the fundamental assumptions a method makes and whether any of those are violated in the dataset of interest

# Cellrank

[`Cellrank`](https://cellrank.readthedocs.io) is a trajectory inference algorithm designed to make the best of both worlds using RNA velcoity and markov modeling. CellRank constructs the Markov Matrix by examining consistency between RNA velocity from splicing dynamics and gene expression changes. Thus, Cellrank can automatically infer the start and terminals of the system, compute lineage probabilities.

Cellrank uses scvelo outputs as inputs for trajectory inference

In [ ]:
# First infer the terminal states of the system in the hematopiesis data using celltype as the key
cr.tl.terminal_states(adata, cluster_key="celltype", weight_connectivities=0.2)

In [ ]:
# Plot the terminal states 
cr.pl.terminal_states(merged_ad)

<b>`Question:`</b> As you see, the inferred terminals are not correct. Why do you think is the reason?

Maybe the inferred terminals are not correct because the RNA velocity results are not consistent with known biology of hematopoiesis. But it could also be that there may not have enough cells or genes to capture the true complexity of the system which could lead to inaccurate terminal state inference.

Lets use the data used in the Cellrank manuscript instead to understand Cellrank better

In [ ]:
# Load the data
adata = sc.read('Users/tgoel/Downloads/Classes/GENOME/GENOME541/hw3/cellrank_panc_diff.h5ad')

In [ ]:
# The dynamics are precomputed
scv.pl.velocity_embedding_stream(adata, basis="umap", legend_fontsize=12, title="", smooth=0.8, min_mass=4)

In [ ]:
# Identify and plot the terminal states in this data
cr.pl.terminal_states(adata)

In [ ]:
# Identify and plot the initial states
cr.tl.initial_states(adata, cluster_key="clusters")
cr.pl.initial_states(adata, discrete=True)
# Note: The error below does not impact the later analyses

In [ ]:
# Compute fate maps
cr.tl.lineages(adata)
cr.pl.lineages(adata, same_plot=False)
cr.pl.lineages(adata, same_plot=True)